# Process TARGET PanCancer Data

Retrieve the downloaded expression data, update gene identifiers to entrez, and curate sample IDs. The script will also identify a balanced hold-out test set to compare projection performance into learned latent spaces across algorithms.

In [14]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
random.seed(1234)

## Read Phenotype Information: skipping until phenotype data found

In [3]:
# path = os.path.join('download', 'TARGET_phenotype.gz')
# pheno_df = pd.read_table(path)

# print(pheno_df.shape)
# pheno_df.head(3)

## Read Probe Mapping Info 
(where chromosomes start and end, saved in a file in downloads)

In [4]:
# path = os.path.join('download', 'gencode.v23.annotation.gene.probemap')
# probe_map_df = pd.read_table(path)

# # Inner merge gene df to get ensembl to entrez mapping
# probe_map_df = probe_map_df.merge(gene_df, how='inner', left_on='gene', right_on='symbol')

# # Mapping to rename gene expression index
# ensembl_to_entrez = dict(zip(probe_map_df.id, probe_map_df.entrez_gene_id))

# print(probe_map_df.shape)
# probe_map_df.head(3)

## Read Gene Expression Data

In [3]:
file = os.path.join('download', 'grein_count_matrix_human.pkl')
expr_df = pd.read_pickle(file)

print(expr_df.shape)
expr_df.head(4)

(27990, 1635)


,gene,gene_symbol,GSM2667747_x,GSM2667748_x,GSM2667749_x,GSM2667750_x,GSM2667751_x,GSM2667752_x,GSM2667753_x,GSM2667754_x,...,GSM2701937,GSM2701938,GSM2701939,GSM2701940,GSM2701941,GSM2701942,GSM2701943,GSM2701944,GSM2701945,GSM2701946
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,17.4324,13.7006,9.8935,11.9023,11.3564,8.9213,16.4894,13.9479,16.8066,7.7069
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,2.0000,2.0443,1.0000,5.5140,7.6165,6.6037,3.0000,7.0000,4.5765,1.0551
2,ENSG00000000419,DPM1,1097.7855,367.2333,316.5226,3895.0588,1536.1375,1084.5276,127.7205,245.1014,...,165.9822,217.9993,114.9995,307.4970,272.9994,301.4232,211.9566,227.9990,264.4285,256.9996
3,ENSG00000000457,SCYL3,601.4702,270.9239,163.6869,1177.2241,873.9074,845.1423,229.8548,134.4694,...,96.6508,84.5137,70.8997,99.7864,145.2329,122.9726,81.5081,79.0215,99.9146,77.1152


## Process gene expression matrix

This involves updating Entrez gene ids, sorting and subsetting

In [4]:
# expr_df = (expr_df
#     .dropna(axis='rows')
#     .reindex(probe_map_df.id)
#     .rename(index=ensembl_to_entrez)
#     .rename(index=old_to_new_entrez)
#     .groupby(level=0).mean()
#     .transpose()
#     .sort_index(axis='rows')
#     .sort_index(axis='columns')
# )

expr_df.index.rename('sample_id', inplace=True)

print(expr_df.shape)
expr_df.head(2)

(27990, 1635)


,gene,gene_symbol,GSM2667747_x,GSM2667748_x,GSM2667749_x,GSM2667750_x,GSM2667751_x,GSM2667752_x,GSM2667753_x,GSM2667754_x,...,GSM2701937,GSM2701938,GSM2701939,GSM2701940,GSM2701941,GSM2701942,GSM2701943,GSM2701944,GSM2701945,GSM2701946
sample_id,,,,,,,,,,,,,,,,,,,,,
0,ENSG00000000003,TSPAN6,1766.0677,420.8206,300.8009,3142.8439,2207.8392,4367.5056,228.5759,650.3287,...,17.4324,13.7006,9.8935,11.9023,11.3564,8.9213,16.4894,13.9479,16.8066,7.7069
1,ENSG00000000005,TNMD,43.9280,18.0091,0.0000,115.8917,63.5936,39.7803,11.7588,1.0153,...,2.0000,2.0443,1.0000,5.5140,7.6165,6.6037,3.0000,7.0000,4.5765,1.0551


## Stratify Balanced Training and Testing Sets in TARGET Gene Expression

Output training and testing gene expression datasets

In [8]:
#strat = pheno_df.set_index('sample_id').reindex(expr_df.index).primary_disease_code

In [7]:
# cancertype_count_df = (
#     pd.DataFrame(strat.value_counts()) #not using value_counts, what number should we use here? 
#     .reset_index()
#     .rename({'index': 'cancertype', 'primary_disease_code': 'n ='}, axis='columns')
# )

# file = os.path.join('data', 'target_sample_counts.tsv') #change which file - do we have a file that works for this?
# cancertype_count_df.to_csv(file, sep='\t', index=False)

# cancertype_count_df

In [9]:
train_df, test_df = train_test_split(expr_df,
                                     test_size=0.1,
                                     random_state=123) #if no stratify defined, should just randomize on its own

In [6]:
print(train_df.shape)
test_df.shape

(25191, 1635)


(2799, 1635)

In [10]:
#save train dataframe to file 
train_file = os.path.join('data', 'train_grein_processed.tsv.gz')
train_df.to_csv(train_file, sep='\t', compression='gzip', float_format='%.3g')

In [11]:
#save test dataframe to file 
test_file = os.path.join('data', 'test_grein_processed.tsv.gz')
test_df.to_csv(test_file, sep='\t', compression='gzip', float_format='%.3g')

In [22]:
train_df

,gene,gene_symbol,GSM2667747_x,GSM2667748_x,GSM2667749_x,GSM2667750_x,GSM2667751_x,GSM2667752_x,GSM2667753_x,GSM2667754_x,...,GSM2701937,GSM2701938,GSM2701939,GSM2701940,GSM2701941,GSM2701942,GSM2701943,GSM2701944,GSM2701945,GSM2701946
sample_id,,,,,,,,,,,,,,,,,,,,,
27744,ENSG00000283046,HTN1,125.4665,140.2465,17.6274,69.7745,73.7733,27.6203,98.1395,12.7103,...,13.4686,11.2211,3.8547,13.0525,4.3899,7.9562,13.4648,15.0952,12.8111,9.5805
9537,ENSG00000153037,SRP19,894.5558,357.1081,187.4946,1421.0479,456.0143,802.6670,124.8560,164.7368,...,246.7201,229.3536,137.9754,276.6840,283.5146,315.5401,315.4701,327.8115,311.3958,287.1789
9769,ENSG00000155115,GTF3C6,851.6143,187.0203,243.4404,2046.5828,476.0532,1977.8905,94.7822,881.8821,...,271.8578,257.8892,179.5595,371.9447,309.9999,370.9075,199.8807,206.8743,194.0005,199.0000
15239,ENSG00000184083,FAM120C,632.3881,285.1058,122.8606,832.8702,957.4550,1495.5920,184.7934,91.7322,...,75.4221,50.0695,31.4341,54.8382,67.9711,66.0159,73.3668,45.4544,63.2615,41.2241
203,ENSG00000008196,TFAP2B,23.2873,2.0018,19.1983,54.7994,19.0940,974.5680,88.7088,88.8378,...,20.4597,24.2683,14.5826,31.6125,12.5990,10.9160,16.7363,16.5086,22.4091,15.6715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15377,ENSG00000184730,APOBR,6.3818,9.1460,0.0000,11.3041,10.1788,9.9309,8.7824,0.0000,...,4218.6053,4018.6666,1948.5265,3348.8270,3218.9786,3721.1712,3864.2544,3813.3692,3034.6335,2921.8278
21602,ENSG00000235346,HLA-G,22.1629,2.8791,12.0961,68.9133,108.9510,0.0000,0.0000,2.0205,...,0.6717,0.0000,1.0915,0.0000,0.0000,0.0000,0.0000,2.8872,4.0845,1.1946
17730,ENSG00000204291,COL15A1,47.6025,26.1180,7.0183,141.7424,56.5391,392.2496,27.0797,59.5957,...,52.6309,26.4790,24.0000,26.1559,32.5506,26.8942,42.6889,45.5209,32.9535,23.5736


## Sort genes based on median absolute deviation and output to file

In [23]:
train_df_mad = abs(train_df - train_df.drop(['sample_id', 'gene', 'gene_symbol'], axis = 1).median()).median()
print(train_df_mad)

TypeError: 'method' object is not subscriptable

In [17]:
# Determine most variably expressed genes and subset
train_df_mad = abs(train_df - train_df.mean()).mean()
mad_genes_df = pd.DataFrame(train_df.median_abs_deviation(axis=0).sort_values(ascending=False)).reset_index()
mad_genes_df.columns = ['gene_id', 'median_abs_deviation']

file = os.path.join('data', 'target_mad_genes.tsv')
mad_genes_df.to_csv(file, sep='\t', index=False)

AttributeError: 'DataFrame' object has no attribute 'median_abs_deviation'